##import

### import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pycaret.regression import *
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import joblib
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from pycaret import *

### 메소드

In [ ]:

# smape 계산 함수
def smape_fnc(true, pred):
  v = 2*abs(pred-true)/(abs(pred)+abs(true))
  output=np.mean(v)*100
  return output

# custom metrics
def custom_metrics(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

### 데이터 불러오기

In [ ]:
path_drive = './csv/'
path = './'

In [ ]:
combined_clust_df = pd.read_csv(path_drive+'combined_clust_df.csv')
test_df = pd.read_csv(path_drive + 'test_df.csv')

In [ ]:
combined_clust_df['요일_sin'] = -np.sin(2 * np.pi * (combined_clust_df['요일']+1)/7.0)
combined_clust_df['요일_cos'] = -np.cos(2 * np.pi * (combined_clust_df['요일']+1)/7.0)

## 테스트용 et

In [ ]:
combined_clust_df.columns.tolist()

In [ ]:
combined_clust_df.describe()

,건물번호,기온(C),풍속(m/s),습도(%),전력소비량(kWh),불쾌지수,불쾌지수_이동평균3,불쾌지수_이동평균5,월,일,...,지식산업센터,할인마트,호텔및리조트,건물유형_라벨,km_cluster,work_time,low_day,particular,요일_sin,요일_cos
count,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,...,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,201840.000000,2.018400e+05,201840.000000
mean,50.687277,25.567179,2.129615,78.776936,2400.864164,75.462416,75.458423,75.453864,6.939358,14.828181,...,0.080856,0.080856,0.080856,6.213436,1.373365,0.550411,0.131272,0.005821,-5.159141e-03,0.010070
std,28.962936,3.774340,1.358261,15.048475,2387.172083,5.448061,5.409489,5.354180,0.792286,8.443628,...,0.272615,0.272615,0.272615,3.655700,1.291963,0.497453,0.337699,0.076076,7.046215e-01,0.709497
min,1.000000,10.100000,0.000000,13.000000,0.000000,50.481070,51.276703,51.735894,6.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-9.749279e-01,-1.000000
25%,25.000000,23.400000,1.200000,69.000000,1080.360000,72.617760,72.660833,72.679919,6.000000,8.000000,...,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,-7.818315e-01,-0.623490
50%,51.000000,25.700000,2.000000,81.000000,1748.520000,76.312960,76.319913,76.342258,7.000000,15.000000,...,0.000000,0.000000,0.000000,6.000000,1.000000,1.000000,0.000000,0.000000,2.449294e-16,0.222521
75%,76.000000,28.100000,2.800000,91.000000,2928.480000,79.401640,79.365643,79.331946,8.000000,22.000000,...,0.000000,0.000000,0.000000,9.000000,3.000000,1.000000,0.000000,0.000000,7.818315e-01,0.900969
max,100.000000,37.100000,13.300000,100.000000,25488.400000,87.789790,87.330393,87.210598,8.000000,31.000000,...,1.000000,1.000000,1.000000,12.000000,3.000000,1.000000,1.000000,1.000000,9.749279e-01,0.900969


Cluster 0: ['건물기타', '대학교', '연구소', '할인마트']  
Cluster 1: ['백화점및아울렛', '호텔및리조트']  
Cluster 2: ['데이터센터', '아파트']  
Cluster 3: ['공공', '병원', '상용', '지식산업센터']

In [ ]:
clust_to_num = {0: [1, 3, 9, 11], 1: [5, 12], 2: [4, 8], 3: [2, 6, 7, 10]}

In [ ]:
model_type = 'et'


#### clust0

In [ ]:
selected_columns = [
    '전력소비량(kWh)',
    '연면적(m2)',
    'work_time',
    '건물번호',
    'holiday',
    'particular',
    '냉방면적(m2)',
    '불쾌지수',
    'low_day',
    '시간',
    '시간_sin',
    '시간_cos',
    '불쾌지수_이동평균3',
    '태양광용량(kW)',
    'ESS저장용량(kWh)',
    '월',
    '기온(C)',
    '요일',
    '요일_sin',
    '요일_cos',
    '일',
    'km_cluster'
]

combined_clust_train_df = combined_clust_df[selected_columns]


clust0_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 0].reset_index(drop=True)

In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust0_train_df, clust0_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# 스태킹

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust0_train_df, clust0_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# pycaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
#stacker_final = finalize_model(stacker)

# 테스트 데이터 예측
y_pred = predict_model(stacker, data=X_test.drop(columns=['km_cluster']))

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:

# pycaret 설정
exp0 = setup(data=clust0_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'0')




In [ ]:
# pycaret 설정
exp0 = setup(data=clust0_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
stacker_final = finalize_model(stacker)

# 모델 저장
save_model(stacker_final, path+'stacker0')

#### clust1

In [ ]:
selected_columns = [
    '전력소비량(kWh)',
    'work_time',
    '연면적(m2)',
    '냉방면적(m2)',
    '시간',
    '시간_sin',
    '시간_cos',
    '건물번호',
    '불쾌지수',
    '월',
    '기온(C)',
    '요일',
    'holiday',
    '불쾌지수_이동평균3',
    '태양광용량(kW)',
    '일',
    'km_cluster'
]

combined_clust_train_df = combined_clust_df[selected_columns]


clust1_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 1].reset_index(drop=True)


In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust1_train_df, clust1_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# 스태킹

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust1_train_df, clust1_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# pycaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
#stacker_final = finalize_model(stacker)

# 테스트 데이터 예측
y_pred = predict_model(stacker, data=X_test.drop(columns=['km_cluster']))

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# pycaret 설정
exp0 = setup(data=clust1_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'1')

In [ ]:
# pycaret 설정
exp0 = setup(data=clust1_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
stacker_final = finalize_model(stacker)

# 모델 저장
save_model(stacker_final, path+'stacker1')

####clust2

In [ ]:
selected_columns2 = ['전력소비량(kWh)',
 '건물번호',
 '냉방면적(m2)',
 '연면적(m2)',
 '시간',
 '불쾌지수',
 '월',
 '시간_sin',
 '불쾌지수_이동평균3',
 'holiday',
 '시간_cos',
 '일',
 '요일',
 '기온(C)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns2]

clust2_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 2].reset_index(drop=True)


In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust2_train_df, clust2_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# 스태킹

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust2_train_df, clust2_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# pycaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
#stacker_final = finalize_model(stacker)

# 테스트 데이터 예측
y_pred = predict_model(stacker, data=X_test.drop(columns=['km_cluster']))

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# pycaret 설정
exp0 = setup(data=clust2_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'2')

In [ ]:
# pycaret 설정
exp0 = setup(data=clust2_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
stacker_final = finalize_model(stacker)

# 모델 저장
save_model(stacker_final, path+'stacker2')

#### clust3

In [ ]:
selected_columns = [
    '전력소비량(kWh)',
    '병원',
    'work_time',
    'low_day',
    '연면적(m2)',
    '냉방면적(m2)',
    'holiday',
    'ESS저장용량(kWh)',
    '건물번호',
    '태양광용량(kW)',
    '불쾌지수',
    '월',
    '불쾌지수_이동평균3',
    '상용',
    '시간',
    '시간_sin',
    '시간_cos',
    '기온(C)',
    '일',
    '요일',
    '요일_sin',
    '요일_cos',
    'particular',
    'km_cluster'
]

combined_clust_train_df = combined_clust_df[selected_columns]

clust3_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 3].reset_index(drop=True)


In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust3_train_df, clust3_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# 스태킹

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust3_train_df, clust3_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# pycaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
#stacker_final = finalize_model(stacker)

# 테스트 데이터 예측
y_pred = predict_model(stacker, data=X_test.drop(columns=['km_cluster']))

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

In [ ]:
# pycaret 설정
exp0 = setup(data=clust3_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'3')

In [ ]:
# pycaret 설정
exp0 = setup(data=clust3_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
stacker_final = finalize_model(stacker)

# 모델 저장
save_model(stacker_final, path+'stacker3')

###예측

In [ ]:
selected_columns0.append('건물유형_라벨')
selected_columns1.append('건물유형_라벨')
selected_columns2.append('건물유형_라벨')
selected_columns3.append('건물유형_라벨')

rm_list = ['전력소비량(kWh)', 'km_cluster']
for value in rm_list:
    selected_columns0.remove(value)
    selected_columns1.remove(value)
    selected_columns2.remove(value)
    selected_columns3.remove(value)


test_df_list = [test_df[selected_columns0],test_df[selected_columns1],test_df[selected_columns2],test_df[selected_columns3]]

In [ ]:

# 예측값을 저장할 컬럼
test_df['answer'] = np.nan

# 각 클러스터 모델에 대해 예측 수행
for cluster, building_labels in clust_to_num.items():
    # 해당 클러스터의 모델 불러오기
    cluster_model = load_model( path+model_type+ str(cluster))
    #cluster_model = load_model( path+'stacker'+ str(cluster))

    # 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
    cluster_test_data = test_df_list[cluster][test_df_list[cluster]['건물유형_라벨'].isin(building_labels)]
    cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
    index_list = cluster_test_data.index.tolist()

    # 불러온 모델로 예측 수행
    predictions = predict_model(cluster_model, cluster_test_data)

    # index_list와 predictions를 이용하여 데이터프레임 생성
    answer_df = pd.DataFrame({'answer': predictions['prediction_label']}, index=index_list)

    # 예측 결과를 원래 데이터프레임에 추가
    test_df.loc[answer_df.index, 'answer'] = answer_df['answer']


test_df

예측 데이터 저장

In [ ]:
sum(test_df['answer'].isna())

In [ ]:
# 데이터 불러오기
df = pd.read_csv(path_drive+'sample_submission.csv')

df['answer'] = test_df['answer']
df.to_csv(path+'submit.csv', index=False)

In [ ]:
len(df[df['answer']<0])

## 군집화

### 각 군집별로 모델 생성(et) +모델별 feature 추가

Cluster 0: ['건물기타', '대학교', '연구소', '할인마트']  
Cluster 1: ['백화점및아울렛', '호텔및리조트']  
Cluster 2: ['데이터센터', '아파트']  
Cluster 3: ['공공', '병원', '상용', '지식산업센터']

In [ ]:
clust_to_num = {0: [1, 3, 9, 11], 1: [5, 12], 2: [4, 8], 3: [2, 6, 7, 10]}

In [ ]:
model_type = 'et'


#### clust0

In [ ]:
selected_columns0 = ['전력소비량(kWh)',
 '연면적(m2)',
 '건물번호',
 '시간_cos',
 '연구소',
 'holiday',
 '냉방면적(m2)',
 'ESS저장용량(kWh)',
 '불쾌지수',
 '대학교',
 '시간',
 '시간_sin',
 '불쾌지수_이동평균3',
 '태양광용량(kW)',
 '월',
 '불쾌지수_이동평균5',
 '기온(C)',
 '요일',
 '일',
 '습도(%)',
 '풍속(m/s)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns0]

clust0_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 0].reset_index(drop=True)



In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust0_train_df, clust0_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

변수 3개 추가 / 제출용

In [ ]:
selected_columns0 = ['전력소비량(kWh)',
 '연면적(m2)',
  '연구소',
  'work_time',
  '건물번호',
  'holiday',
  'particular',
  '냉방면적(m2)',
  '시간_cos',
  '대학교',
  '불쾌지수',
  'low_day',
  '불쾌지수_이동평균3',
  '시간',
  '태양광용량(kW)',
  'ESS저장용량(kWh)',
  '월',
  '불쾌지수_이동평균5',
  '시간_sin',
  '기온(C)',
  '요일',
  '일',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns0]

clust0_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 0].reset_index(drop=True)



In [ ]:

# pycaret 설정
exp0 = setup(data=clust0_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'0')




#### clust1

In [ ]:
selected_columns1 = ['전력소비량(kWh)',
 '시간_cos',
 '냉방면적(m2)',
 '시간',
 '연면적(m2)',
 '시간_sin',
 '건물번호',
 '불쾌지수_이동평균3',
 '월',
 '불쾌지수',
 'holiday',
 '요일',
 '불쾌지수_이동평균5',
 '태양광용량(kW)',
 '일',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns1]

clust1_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 1].reset_index(drop=True)



In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust1_train_df, clust1_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

변수 3개 추가 / 제출용

In [ ]:
selected_columns1 = ['전력소비량(kWh)',
'work_time', '연면적(m2)', '냉방면적(m2)', '시간', '건물번호', '시간_cos', '불쾌지수', '월', '기온(C)', 'holiday', '시간_sin', '불쾌지수_이동평균3', '요일', '불쾌지수_이동평균5', '일', '습도(%)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns1]

clust1_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 1].reset_index(drop=True)



In [ ]:
# pycaret 설정
exp0 = setup(data=clust1_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'1')

####clust2

In [ ]:
selected_columns2 = ['전력소비량(kWh)',
 '건물번호',
 '냉방면적(m2)',
 '불쾌지수_이동평균5',
 '연면적(m2)',
 '시간',
 '불쾌지수',
 '월',
 '시간_sin',
 '불쾌지수_이동평균3',
 'holiday',
 '시간_cos',
 '일',
 '습도(%)',
 '요일',
 '기온(C)',
 '풍속(m/s)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns2]

clust2_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 2].reset_index(drop=True)


In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust2_train_df, clust2_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

변수 3개 추가 / 제출용

In [ ]:
selected_columns2 = ['전력소비량(kWh)',
 '건물번호',
 '냉방면적(m2)',
 '불쾌지수_이동평균5',
 '연면적(m2)',
 '시간',
 '불쾌지수',
 '월',
 '시간_sin',
 '불쾌지수_이동평균3',
 'holiday',
 '시간_cos',
 '일',
 '습도(%)',
 '요일',
 '기온(C)',
 '풍속(m/s)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns2]

clust2_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 2].reset_index(drop=True)


In [ ]:
# pycaret 설정
exp0 = setup(data=clust2_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'2')

#### clust3

In [ ]:
selected_columns3 = ['전력소비량(kWh)',
 '병원',
 '시간_cos',
 '연면적(m2)',
 'holiday',
 '냉방면적(m2)',
 '건물번호',
 '태양광용량(kW)',
 'ESS저장용량(kWh)',
 '불쾌지수',
 '시간_sin',
 '상용',
 '월',
 '불쾌지수_이동평균3',
 '시간',
 '불쾌지수_이동평균5',
 '요일',
 '기온(C)',
 '일',
 '습도(%)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns3]

clust3_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 3].reset_index(drop=True)



In [ ]:
# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust3_train_df, clust3_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# PyCaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 최종 모델 생성
et_model0 = create_model(model_type)
final_et_model0 = finalize_model(et_model0)

# 테스트 데이터 예측
y_pred = predict_model(final_et_model0, data=X_test)

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")

변수 3개 추가 / 제출용

In [ ]:
selected_columns3 = ['전력소비량(kWh)',
'work_time',
 '연면적(m2)',
 '냉방면적(m2)',
 '시간',
 '건물번호',
 '시간_cos',
 '불쾌지수',
 '월',
 '기온(C)',
 'holiday',
 '시간_sin',
 '불쾌지수_이동평균3',
 '요일',
 '불쾌지수_이동평균5',
 '일',
 '습도(%)',
 '풍속(m/s)',
 '태양광용량(kW)',
'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns3]

clust3_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 3].reset_index(drop=True)



In [ ]:
# pycaret 설정
exp0 = setup(data=clust3_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model0 = create_model(model_type)

# 최종 모델 생성
final_et_model0 = finalize_model(et_model0)

# 모델 저장
save_model(final_et_model0, path+model_type+'3')

###예측

In [ ]:
selected_columns0.append('건물유형_라벨')
selected_columns1.append('건물유형_라벨')
selected_columns2.append('건물유형_라벨')
selected_columns3.append('건물유형_라벨')

rm_list = ['전력소비량(kWh)', 'km_cluster']
for value in rm_list:
    selected_columns0.remove(value)
    selected_columns1.remove(value)
    selected_columns2.remove(value)
    selected_columns3.remove(value)


test_df_list = [test_df[selected_columns0],test_df[selected_columns1],test_df[selected_columns2],test_df[selected_columns3]]

In [ ]:

# 예측값을 저장할 컬럼
test_df['answer'] = np.nan

# 각 클러스터 모델에 대해 예측 수행
for cluster, building_labels in clust_to_num.items():
    # 해당 클러스터의 모델 불러오기
    cluster_model = load_model( path+model_type++ str(cluster))

    # 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
    cluster_test_data = test_df_list[cluster][test_df_list[cluster]['건물유형_라벨'].isin(building_labels)]
    cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
    index_list = cluster_test_data.index.tolist()

    # 불러온 모델로 예측 수행
    predictions = predict_model(cluster_model, cluster_test_data)

    # index_list와 predictions를 이용하여 데이터프레임 생성
    answer_df = pd.DataFrame({'answer': predictions['prediction_label']}, index=index_list)

    # 예측 결과를 원래 데이터프레임에 추가
    test_df.loc[answer_df.index, 'answer'] = answer_df['answer']


test_df

예측 데이터 저장

In [ ]:
sum(test_df['answer'].isna())

In [ ]:
# 데이터 불러오기
df = pd.read_csv(path_drive+'sample_submission.csv')

df['answer'] = test_df['answer']
df.to_csv(path+'submit.csv', index=False)

In [ ]:
len(df[df['answer']<0])